## Name: Mitkumar G. Patel
## Roll No:21BCP285
## Course:Computer Vision Lab
## Course Code: 20CP407P
## Project: "Exploring Image Processing Techniques: A Gradio-Powered Application with Interactive Controls"

#### Follow instructions for share project link:
For using the ngrok : first run the application on localhost.<br>
go to folder where the ngrok.exe is extracted -> open cmd there-> type command<br>
Command: ngrok http 7860 then wait for output link given by the ngrok in cmd -> share it to other user.

In [1]:
import gradio as gr
import cv2
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans

C:\Users\Mit\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#load the image
def load_image(image_input):
    return image_input  

load_image_interface = gr.Interface(fn=load_image, inputs="image", outputs="image")

In [3]:
# Apply binarization
def binarization(image):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary_img
    
binary_image_interface=gr.Interface(fn=binarization,inputs="image", outputs="image")

In [4]:
# Apply noise reduction
def noise_reduction(image):
    blurred_img = cv2.GaussianBlur(image, (5, 5), 0)
    return blurred_img

blurred_img_interface=gr.Interface(fn=noise_reduction,inputs="image",outputs="image")


In [5]:
# Resize the image
def Resize(image,width,height):
    resized_img=cv2.resize(image,(int(width),int(height)))
    return resized_img

resize_image_interface = gr.Interface(fn=Resize, 
                                    inputs=[gr.Image(),gr.Number(label='Width'),gr.Number(label='Height')],
                                    outputs='image')


In [6]:
def Reduce_size_quality(image, target_size, unit):
    # Convert the target size to KB
    if unit == "MB":
        target_size *= 1024  # Convert MB to KB
    
    # Compress the image and return
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 95]
    
    # Convert the image from RGB to BGR before saving
    is_success, buffer = cv2.imencode(".jpg", cv2.cvtColor(image, cv2.COLOR_RGB2BGR), encode_param)
    
    # Start reducing the quality
    for quality in range(95, 0, -5):
        # Set JPEG quality
        encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
        is_success, buffer = cv2.imencode(".jpg", cv2.cvtColor(image, cv2.COLOR_RGB2BGR), encode_param)
        
        # Check file size in KB
        img_size_kb = len(buffer) / 1024
        if img_size_kb <= target_size:
            break
    
    # Convert buffer back to a PIL Image
    img_compressed = Image.open(io.BytesIO(buffer))
    return img_compressed

# Create Gradio interface for reducing image size
reduce_size_interface = gr.Interface(
    fn=Reduce_size_quality, 
    inputs=[gr.Image(), gr.Number(label="Target Size"), gr.Radio(choices=["KB", "MB"], label="Unit")], 
    outputs="image"
)


In [7]:
# Image Segmentation
def suggest_clusters(image):
    # Convert image to feature vector
    pixels = image.reshape(-1, 3)
    
    # Using elbow method to suggest optimal k
    distortions = []
    K = range(1, 10)
    for k in K:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(pixels)
        distortions.append(kmeans.inertia_)
    
    # Find elbow point
    suggested_k = 3  # default
    for i in range(1, len(distortions)-1):
        if (distortions[i-1] - distortions[i]) / (distortions[i] - distortions[i+1]) < 2:
            suggested_k = i + 1
            break
            
    return suggested_k

def image_segmentation(image, k=None):
    if k is None:
        k = suggest_clusters(image)
    
    pixels = image.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(pixels)
    centers = kmeans.cluster_centers_
    segmented = centers[labels].reshape(image.shape)
    return segmented.astype(np.uint8)

# Create Gradio interface for image segmentation
segmentation_interface = gr.Interface(
    fn=image_segmentation,
    inputs=[gr.Image(), gr.Number(label="Number of Clusters (K)", value=None)],
    outputs="image"
)

In [8]:
#Image Transformations
def translate_image(image, tx=50, ty=50):
    rows, cols = image.shape[:2]
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    return cv2.warpAffine(image, M, (cols, rows))

def rotate_image(image, angle=90):
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
    return cv2.warpAffine(image, M, (cols, rows))

def reflect_image(image, axis='x'):
    return cv2.flip(image, 0 if axis == 'x' else 1)

def scale_image(image, sx=1.5, sy=1.5, axis='both'):
    rows, cols = image.shape[:2]
    if axis == 'x':
        sy = 1.0
    elif axis == 'y':
        sx = 1.0
    M = np.float32([[sx, 0, 0], [0, sy, 0]])
    return cv2.warpAffine(image, M, (int(cols*sx), int(rows*sy)))

def shear_image(image, shx=0.5, shy=0.5, axis='both'):
    rows, cols = image.shape[:2]
    if axis == 'x':
        shy = 0
    elif axis == 'y':
        shx = 0
    M = np.float32([[1, shx, 0], [shy, 1, 0]])
    return cv2.warpAffine(image, M, (cols, rows))

# Create Gradio interfaces for transformations
translation_interface = gr.Interface(
    fn=translate_image,
    inputs=[gr.Image(), gr.Number(label="TX", value=50), gr.Number(label="TY", value=50)],
    outputs="image"
)

rotation_interface = gr.Interface(
    fn=rotate_image,
    inputs=[gr.Image(), gr.Number(label="Angle", value=90, minimum=0, maximum=360)],
    outputs="image"
)

reflection_interface = gr.Interface(
    fn=reflect_image,
    inputs=[gr.Image(), gr.Radio(choices=['x', 'y'], label="Reflection Axis", value='x')],
    outputs="image"
)

scaling_interface = gr.Interface(
    fn=scale_image,
    inputs=[
        gr.Image(),
        gr.Number(label="Scale X", value=1.5),
        gr.Number(label="Scale Y", value=1.5),
        gr.Radio(choices=['x', 'y', 'both'], label="Scaling Axis", value='both')
    ],
    outputs="image"
)

shearing_interface = gr.Interface(
    fn=shear_image,
    inputs=[
        gr.Image(),
        gr.Number(label="Shear X", value=0.5),
        gr.Number(label="Shear Y", value=0.5),
        gr.Radio(choices=['x', 'y', 'both'], label="Shearing Axis", value='both')
    ],
    outputs="image"
)

In [9]:
# Edge Detection
def edge_detection_sobel(image, direction='both'):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    if direction == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
    elif direction == 'y':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
    else:  # both
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
        sobel = np.sqrt(np.square(sobelx) + np.square(sobely))
    
    return np.uint8(np.absolute(sobel))

def edge_detection_canny(image, low_threshold=100, high_threshold=200):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.Canny(gray, low_threshold, high_threshold)

# Create Gradio interfaces for edge detection
sobel_interface = gr.Interface(
    fn=edge_detection_sobel,
    inputs=[gr.Image(), gr.Radio(choices=['x', 'y', 'both'], label="Sobel Direction", value='both')],
    outputs="image"
)

canny_interface = gr.Interface(
    fn=edge_detection_canny,
    inputs=[
        gr.Image(),
        gr.Number(label="Low Threshold",value=100),
        gr.Number(label="High Threshold", value=200)
    ],
    outputs="image"
)

In [11]:
# Main processing function
def apply_function(image, selection, width=None, height=None, target_size=None, unit=None,
                  k=None, transform_type=None, tx=None, ty=None, angle=None,
                  reflection_axis=None, scale_axis=None, sx=None, sy=None,
                  shear_axis=None, shx=None, shy=None, edge_type=None,
                  sobel_direction=None, low_threshold=None, high_threshold=None):
    if selection == "Resize Image":
        return Resize(image, width, height)
    elif selection == "Reduce Image Size":
        return Reduce_size_quality(image, target_size, unit)
    elif selection == "Convert to Binary":
        return binarization(image)
    elif selection == "Noise Reduction":
        return noise_reduction(image)
    elif selection == "Image Segmentation":
        return image_segmentation(image, k)
    elif selection == "Image Transformation":
        if transform_type == "Translation":
            return translate_image(image, tx, ty)
        elif transform_type == "Rotation":
            return rotate_image(image, angle)
        elif transform_type == "Reflection":
            return reflect_image(image, reflection_axis)
        elif transform_type == "Scaling":
            return scale_image(image, sx, sy, scale_axis)
        elif transform_type == "Shearing":
            return shear_image(image, shx, shy, shear_axis)
    elif selection == "Edge Detection":
        if edge_type == "Sobel":
            return edge_detection_sobel(image, sobel_direction)
        elif edge_type == "Canny":
            return edge_detection_canny(image, low_threshold, high_threshold)
    return image

# Create the main Gradio interface
def create_gradio_interface():
    with gr.Blocks() as app:
        gr.Markdown("## CV Lab Project: Advanced Image Processing App")
        
        with gr.Row():
            image_input = gr.Image(label="Upload Image")
            output_image = gr.Image(label="Output Image")
            
        with gr.Column():
            # Main function selector
            function_selector = gr.Radio(
                choices=[
                    "Resize Image", "Reduce Image Size", "Binarization",
                    "Noise Reduction", "Image Segmentation", "Image Transformation",
                    "Edge Detection"
                ],
                label="Choose a Function",
                value="Resize Image"
            )
            
            # Original inputs
            width_input = gr.Number(label="Width", value=256, visible=False)
            height_input = gr.Number(label="Height", value=256, visible=False)
            target_size_input = gr.Number(label="Target Size", value=100, visible=False)
            unit_input = gr.Radio(choices=["KB", "MB"], label="Unit", value="KB", visible=False)
            
            # Segmentation inputs
            k_input = gr.Number(label="Number of Clusters (K)", visible=False)
            
            # Transformation inputs
            transform_type = gr.Radio(
                choices=["Translation", "Rotation", "Reflection", "Scaling", "Shearing"],
                label="Transform Type",
                visible=False
            )
            
            with gr.Column(visible=False) as transform_params:
                tx_input = gr.Number(label="TX", value=50)
                ty_input = gr.Number(label="TY", value=50)
                angle_input = gr.Number(label="Rotation Angle", value=90)
                reflection_axis = gr.Radio(choices=['x', 'y'], label="Reflection Axis")
                scale_axis = gr.Radio(choices=['x', 'y', 'both'], label="Scale Axis")
                sx_input = gr.Number(label="Scale X", value=1.5)
                sy_input = gr.Number(label="Scale Y", value=1.5)
                shear_axis = gr.Radio(choices=['x', 'y', 'both'], label="Shear Axis")
                shx_input = gr.Number(label="Shear X", value=0.5)
                shy_input = gr.Number(label="Shear Y", value=0.5)
            
            # Edge detection inputs
            edge_type = gr.Radio(
                choices=["Sobel", "Canny"],
                label="Edge Detection Type",
                visible=False
            )
            sobel_direction = gr.Radio(
                choices=['x', 'y', 'both'],
                label="Sobel Direction",
                visible=False
            )
            with gr.Column(visible=False) as canny_params:
                low_threshold = gr.Number(label="Low Threshold", value=100)
                high_threshold = gr.Number(label="High Threshold", value=200)
            
            submit_button = gr.Button("Submit")
        
              
        # Function to toggle visibility of inputs
        def toggle_inputs(selection):
            outputs = {
                "width": False, "height": False,
                "target_size": False, "unit": False,
                "k": False,
                "transform_type": False,
                "transform_params": False,
                "edge_type": False,
                "sobel_direction": False,
                "canny_params": False
            }
            
            if selection == "Resize Image":
                outputs.update({"width": True, "height": True})
            elif selection == "Reduce Image Size":
                outputs.update({"target_size": True, "unit": True})
            elif selection == "Image Segmentation":
                outputs.update({"k": True})
            elif selection == "Image Transformation":
                outputs.update({"transform_type": True, "transform_params": True})
            elif selection == "Edge Detection":
                outputs.update({"edge_type": True, "sobel_direction": True, "canny_params": True})
            
            return [
                gr.update(visible=outputs["width"]),
                gr.update(visible=outputs["height"]),
                gr.update(visible=outputs["target_size"]),
                gr.update(visible=outputs["unit"]),
                gr.update(visible=outputs["k"]),
                gr.update(visible=outputs["transform_type"]),
                gr.update(visible=outputs["transform_params"]),
                gr.update(visible=outputs["edge_type"]),
                gr.update(visible=outputs["sobel_direction"]),
                gr.update(visible=outputs["canny_params"])
            ]

        # Function to toggle transformation parameters
        def toggle_transform_params(transform):
            is_translation = transform == "Translation"
            is_rotation = transform == "Rotation"
            is_reflection = transform == "Reflection"
            is_scaling = transform == "Scaling"
            is_shearing = transform == "Shearing"
            
            return [
                gr.update(visible=is_translation),  # tx
                gr.update(visible=is_translation),  # ty
                gr.update(visible=is_rotation),     # angle
                gr.update(visible=is_reflection),   # reflection_axis
                gr.update(visible=is_scaling),      # scale_axis
                gr.update(visible=is_scaling),      # sx
                gr.update(visible=is_scaling),      # sy
                gr.update(visible=is_shearing),     # shear_axis
                gr.update(visible=is_shearing),     # shx
                gr.update(visible=is_shearing)      # shy
            ]

        # Function to toggle edge detection parameters
        def toggle_edge_params(edge_type):
            is_sobel = edge_type == "Sobel"
            is_canny = edge_type == "Canny"
            
            return [
                gr.update(visible=is_sobel),    # sobel_direction
                gr.update(visible=is_canny),     # canny_params
            ]

        # Link the function selector to toggle inputs
        function_selector.change(
            toggle_inputs,
            inputs=function_selector,
            outputs=[
                width_input, height_input,
                target_size_input, unit_input,
                k_input,
                transform_type,
                transform_params,
                edge_type,
                sobel_direction,
                canny_params
            ]
        )

        # Link transform type to toggle transform parameters
        transform_type.change(
            toggle_transform_params,
            inputs=transform_type,
            outputs=[
                tx_input, ty_input,
                angle_input,
                reflection_axis,
                scale_axis, sx_input, sy_input,
                shear_axis, shx_input, shy_input
            ]
        )

        # Link edge detection type to toggle edge parameters
        edge_type.change(
            toggle_edge_params,
            inputs=edge_type,
            outputs=[sobel_direction, canny_params]
        )

        # Submit button click event
        submit_button.click(
            fn=apply_function,
            inputs=[
                image_input, function_selector,
                width_input, height_input,
                target_size_input, unit_input,
                k_input,
                transform_type, tx_input, ty_input,
                angle_input, reflection_axis,
                scale_axis, sx_input, sy_input,
                shear_axis, shx_input, shy_input,
                edge_type, sobel_direction,
                low_threshold, high_threshold
            ],
            outputs=output_image
        )

    return app


gradio_app = create_gradio_interface()
gradio_app.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
